In [44]:
import os 
import netCDF4 as nc
import xarray as xr
import matplotlib.pyplot as plt
import cartopy
import numpy as np

In [3]:
slp_file = '/localdrive/drive10/jj/mdtf/inputdata/model/QBOi.EXP1.AMIP.001/6hr/QBOi.EXP1.AMIP.001.SLP.6hr.nc'

In [68]:
ncid = nc.Dataset(slp_file)
calendar = ncid['time'].getncattr('calendar')
ncid.close()

in_ds = xr.open_dataset(slp_file)

# Getting the necessary data from the full year list file
time = in_ds.time.values
slp = in_ds.SLP
lat = in_ds.lat.values
lon = in_ds.lon.values

# creating the year_list to chunk out the yearly sections of the files
year_list = np.array([i.year for i in time])

In [78]:
year_range = [1979, 1979]
sYear = year_range[0]
eYear = year_range[1]

for year in range(sYear, eYear+1): 
    ind = (year_list == year)
    slp_sel = slp[ind, :, :]
    
    out_file= f'./slp.{year:04d}.nc'
    time = np.arange(0, np.sum(ind)*6, 6)
    
    out_ds = xr.Dataset(
        {'slp': (('time', 'lat', 'lon'), slp_sel)}, 
        coords={
            'time': time, 
            'lat': lat, 
            'lon': lon
        }
    )
    
    out_ds.slp.attrs['units'] = 'mb'
    
    out_ds.time.attrs['delta_t'] = "0000-00-00 06:00:00";
    out_ds.time.attrs['units'] = f"hours since {year:04d}-01-01 00:00:00";
    if (calendar == 'noleap'):
        out_ds.time.attrs['calendar'] = '365_day'
    else:
        out_ds.time.attrs['calendar'] = calendar
        
    out_ds.lon.attrs['long_name'] = 'longitude'
    out_ds.lon.attrs['standard_name'] = 'longitude'
    out_ds.lon.attrs['units'] = 'degrees_east'
    out_ds.lon.attrs['axis'] = 'X'
    
    out_ds.lat.attrs['long_name'] = 'latitude'
    out_ds.lat.attrs['standard_name'] = 'latitude'
    out_ds.lat.attrs['units'] = 'degrees_north'
    out_ds.lat.attrs['axis'] = 'Y'
            
#     out_ds.to_netcdf(out_file)

1460


In [38]:
# Original data of SLP FILES 
tmp_file = '/localdrive/drive6/erai/slp/slp.1979.nc'
tmp_ds = xr.open_dataset(tmp_file)
print(tmp_ds)

<xarray.Dataset>
Dimensions:  (lat: 121, lon: 240, time: 1460)
Coordinates:
  * lon      (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 352.5 354.0 355.5 357.0 358.5
  * lat      (lat) float64 90.0 88.5 87.0 85.5 84.0 ... -85.5 -87.0 -88.5 -90.0
  * time     (time) datetime64[ns] 1979-01-01 ... 1979-12-31T18:00:00
Data variables:
    slp      (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.5.3 (http://code.zmaw.de/p...
    Conventions:  CF-1.0
    history:      Mon Dec 17 13:57:37 2012: cdo setvar,msl SLP_1979.nc NEW_SL...
    institution:  European Centre for Medium-Range Weather Forecasts
    CDO:          Climate Data Operators version 1.5.3 (http://code.zmaw.de/p...
